In [ ]:
%matplotlib inline

# Omori law statistics

The modified Omori law (*Utsu, 1961*) describes the power-law decay of aftershock rates following a mainshock with magnitude $M_m$:

$$N(t,M \geq M_c) = K(t+c)^{-p}$$

- $N$ : cumulative number of aftershocks
- $t$ : elapsed time since mainshock
- $M_c$: completeness magnitude
- $K$ : productivity (initial slope), depends on $M_m$ and $M_c$
- $p$ : power-law exponent, independent of $M_m$ and $M_c$ (0.6 - 2.5 according to worldwide survey, with median ~1.1)
- $c$ : small positive constant, interpreted as delay of earliest part of the sequence (0.01 - 1 days)

Based on the following papers:
- Utsu, T., Ogata, Y., Matsu'ura, R. (1995), "The Centenary of the Omori Formula for a Decay Law of Aftershock Activity", *J. Phys. Earth*, **43**, 1-33 (https://www.jstage.jst.go.jp/article/jpe1952/43/1/43_1_1/_pdf)

In [ ]:
import datetime
import pylab
import numpy as np
import mapping.layeredbasemap as lbm
import eqcatalog
import hazard.rshalib as rshalib

Read earthquake sequence from database

In [ ]:
start_date = datetime.date(2021, 1, 1)
cat = eqcatalog.rob.query_local_eq_catalog(start_date=start_date, region=(6.0, 6.4, 50.58, 50.875))
cat.print_info()

In [ ]:
cat.plot_map(resolution='h')

### Declustering analysis

Check if different declustering windows/methods identify sequence as 1 cluster. Note that all declustering methods are based on $M_W$, so we need a conversion from $M_L$.

In [ ]:
dc_method = eqcatalog.declustering.LinkedWindowMethod()

In [ ]:
Mrelation = {'ML': 'GruenthalEtAl2009'}
lw_dc_results = {}
for dc_window_name in ('GardnerKnopoff1974', 'Uhrhammer1986', 'Gruenthal2009'):
    print(dc_window_name)
    dc_window = eqcatalog.declustering.get_window_by_name(dc_window_name)
    dc_result = dc_method.analyze_clusters(cat, dc_window, Mrelation,
                                          ignore_location_errors=True)
    dc_result.print_info()
    lw_dc_results[dc_window_name] = dc_result
    print()

In [ ]:
dc_method = eqcatalog.declustering.ReasenbergMethod()
dc_window = eqcatalog.declustering.Reasenberg1985Window(dsigma=30, xmeff=0.7)
dc_result = dc_method.analyze_clusters(cat, Mrelation, dc_window, ignore_location_errors=False)
print('Reasenberg1985')
dc_result.print_info()

In [ ]:
dc_result.get_unclustered_events().print_list()

### Omori-law fitting

Determine completeness magnitude Mc

In [ ]:
completeness = cat.get_uniform_completeness(0, Mtype='ML')
seq_mfd = cat.get_incremental_mfd(0, 3, 0.2, Mtype='ML', completeness=completeness)
seq_mfd.plot()

Mc can be estimated as 0.7 (0.3 if you are optimistic)

In [ ]:
Mc = 0.3
#Mc = 0.7

Estimate parameters K, c, p of Omori law based on elapsed times since mainshock

In [ ]:
mainshock = cat.get_event_by_id(11630)
Mm = mainshock.ML
mainshock.print_info()

In [ ]:
cluster = dc_result.get_cluster_by_eq(mainshock)
aftershocks = cluster.get_aftershocks()
aftershocks.print_info()

In [ ]:
cc_aftershocks = aftershocks.subselect(Mmin=Mc, Mtype='ML')
print(len(cc_aftershocks))

In [ ]:
as_time_deltas = cc_aftershocks.get_time_deltas(mainshock.datetime)
as_time_deltas = eqcatalog.time.fractional_time_delta(as_time_deltas, 'D')
as_time_deltas

In [ ]:
pylab.plot(as_time_deltas, np.arange(len(cc_aftershocks))+1)

Fit full sequence

In [ ]:
#(K1, c1, p1) = eqcatalog.omori.estimate_omori_params(as_time_deltas)
(K1, c1, p1), _, _ = eqcatalog.omori.OmoriLaw.fit_cumulative(as_time_deltas, np.arange(len(as_time_deltas)))
print(K1, c1, p1)

Fit first 12 days

In [ ]:
(K2, c2, p2) = eqcatalog.omori.estimate_omori_params(as_time_deltas[as_time_deltas < 12])
print(K2, c2, p2)

Note that value of K depends on completeness magnitude and mainshock magnitude, so these are inherent properties of the Omori law!

Define Omori law, including completeness magnitude and mainshock magnitude. Default time unit is days.

In [ ]:
omlaw1 = eqcatalog.omori.OmoriLaw(K1, c1, p1, Mc, Mm)
omlaw2 = eqcatalog.omori.OmoriLaw(K2, c2, p2, Mc, Mm)

In [ ]:
today = datetime.date.today()
num_days = (today - start_date).days
x_values = np.linspace(0, num_days, 50)
marker_sizes = ((np.array([eq.ML for eq in cc_aftershocks]) - Mc) + 2)**2
#observed_cluster_idxs = [0,1] * (len(cc_aftershocks) // 2)
observed_cluster_idxs = [lw_dc_results['GardnerKnopoff1974'].get_cluster_by_eq(eq).ID
                         for eq in cc_aftershocks]
unique_cluster_idxs = np.unique(observed_cluster_idxs)
cm = pylab.cm.rainbow
colors = [cm(i) for i in np.linspace(0, 1, len(unique_cluster_idxs))]
label = 'Omori fit ($M_c=%.1f$)' % Mc
omlaw1.plot_cumulative(x_values, observed_delta_t=as_time_deltas, xscaling='lin',
                      observed_marker_sizes=marker_sizes, observed_marker='o',
                      observed_cluster_colors=colors,
                      observed_cluster_idxs=observed_cluster_idxs, label=label)

In [ ]:
cc_aftershocks[[4,10,16]].print_list()

In [ ]:
omlaw1.plot_rate(x_values)

### Probabilities and predictions

Assuming aftershock occurrence follows a (non-stationary) **Poisson** probability model (see separate notebook), it is possible to compute probabilities and make some predictions. All predictions are valid for $M \geq Mc$

Number of aftershocks

In [ ]:
end_time = 15
omlaw2.get_num_aftershocks(end_time)

In [ ]:
start_time = 28
end_time = start_time + 7
omlaw1.get_num_aftershocks(end_time, start_time)

Probability of exactly $n$ earthquakes

In [ ]:
start_time = 5
end_time = 6
for n in range(6):
    p = omlaw2.get_prob_n_aftershocks(n, end_time, start_time)
    print('n=%d: p=%.4f' % (n, p))

Probability of $\geq 1$ earthquakes ($=1-P(0)$)

In [ ]:
start_time = 28
end_time = start_time + 7
omlaw1.get_prob_one_or_more_aftershocks(end_time, start_time)

Duration

In [ ]:
n = 15
omlaw1.get_time_delta_for_n_aftershocks(n)

In [ ]:
omlaw1.get_time_delta_for_n_aftershocks(n, delta_t1=2)

In [ ]:
omlaw1.get_interaction_time(0.95, 30)

To compute aftershock duration, we need to know the background rate. We estimate this from the entire catalog since 1985, convert to daily rate, and correct for area

In [ ]:
full_cat = eqcatalog.read_named_catalog('ROB', verbose=False).subselect(start_date=1985)
full_cat.print_info()

Determine Gutenberg-Richter relation for entire catalog.

In [ ]:
completeness = full_cat.get_uniform_completeness(0, Mtype='ML')
cat_imfd = full_cat.get_incremental_mfd(1.75, 5, 0.2, Mtype='ML', completeness=completeness)
cat_tmfd = full_cat.get_estimated_mfd(1.75, 5, 0.2, Mtype='ML', completeness=completeness)
cat_tmfd.min_mag = Mc
print(cat_tmfd)
rshalib.mfd.plot_mfds([cat_imfd, cat_tmfd], labels=['Observed', 'GRT fit'])

In [ ]:
area_factor = (lbm.PolygonData.from_bbox(full_cat.get_region()).get_area()
               / lbm.PolygonData.from_bbox(cat.get_region()).get_area())
print(area_factor)

In [ ]:
background_rate = cat_tmfd.get_cumulative_rates()[0]
background_rate /= 365
background_rate /= area_factor
print(background_rate)

In [ ]:
#background_rate = 1./30
omlaw2.get_aftershock_duration(background_rate)

We can do some predictions, but not about the magnitude distribution!

In order to do that, we need to combine the Omori law with the Gutenberg-Richter relation (*Reasenberg & Jones, 1989, 1994*). This involves converting K to the magnitude-independent productivity parameter $A$, which is thought to be characteristic of a region:

$$A = \log_{10}(K) - b (M_m - M_c)$$

where $b$ is the b-value of the Gutenberg-Richter relation (log10 notation)

Note that it would be better to convert ML to MW for this exercise

In [ ]:
as_imfd = cc_aftershocks.get_incremental_mfd(0.7, Mm, Mtype='ML', completeness=completeness)
as_tmfd = cc_aftershocks.get_estimated_mfd(0.7, Mm, Mtype='ML', completeness=completeness)
print(as_tmfd)
rshalib.mfd.plot_mfds([as_imfd, as_tmfd], labels=['Observed', 'GR fit'])

 Note that b-value is similar to that of the full catalog!

In [ ]:
b_value = as_tmfd.b_val
gr_omlaw = omlaw2.to_gr_omori_law(b_value)

In [ ]:
gr_omlaw.K

Now we can compute probabilities for higher Mc, simply by changing the Mc property of the Base10GROmoriLaw instance

In [ ]:
gr_omlaw.Mc = 4.5
gr_omlaw.K

In [ ]:
start_time = 13
end_time = start_time + 30
gr_omlaw.get_prob_one_or_more_aftershocks(end_time, start_time)

Let's compare this to the background probability, derived from the catalog MFD

In [ ]:
tau = cat_tmfd.get_return_periods()[cat_tmfd.get_magnitude_index(gr_omlaw.Mc)]
tau *= 365
print(tau)

Over the entire area covered by the catalog:

In [ ]:
poisson_tau = rshalib.poisson.PoissonTau(tau)
poisson_tau.get_prob_one_or_more(30)

In the area of the Rott sequence:

In [ ]:
poisson_tau.tau *= area_factor
poisson_tau.get_prob_one_or_more(30)

So the sequence has indeed increased the probability of larger earthquakes (but this probability will decrease again over time)!

What is the probability of an earthquake with $M_L \geq 2.6$ (mainshock magnitude) based on the first 12 days?

In [ ]:
gr_omlaw2 = omlaw2.to_gr_omori_law(b_value)
gr_omlaw2.plot_cumulative(x_values, observed_delta_t=as_time_deltas, xscaling='lin')

In [ ]:
gr_omlaw2.Mc = Mm
start_time = 12
end_time = start_time + 30
gr_omlaw2.get_prob_one_or_more_aftershocks(end_time, start_time)

If we assume that the 1911 Eifel sequence (mainshock magnitude $M_L=4.5$) did have the same productivity, what is its duration for $M_L \geq 2.0$ aftershocks?

In [ ]:
gr_omlaw2.Mm = 4.5
gr_omlaw2.Mc = 2.0

In [ ]:
background_rate = cat_tmfd.get_cumulative_rates()[cat_tmfd.get_magnitude_index(gr_omlaw.Mc)]
background_rate /= (365. * area_factor)
background_rate

In [ ]:
duration = gr_omlaw.get_aftershock_duration(background_rate) / 365.
print('Duration: %.2f years' % duration)

### Random simulation

We can also simulate aftershock sequences or their properties

In [ ]:
start_time = 13
end_time = start_time + 30
num_samples = 100
gr_omlaw.get_random_num_aftershocks(end_time, start_time, num_samples)

In [ ]:
gr_omlaw.Mc = 2
gr_omlaw.get_random_num_aftershocks(end_time, start_time, num_samples)

In [ ]:
duration = 100
gr_omlaw.get_random_time_deltas(duration)

In [ ]:
num_samples = 20
gr_omlaw.get_random_magnitudes(num_samples)

In [ ]:
duration = 100
Mmax = 6.0
for event in gr_omlaw.gen_aftershock_sequence(duration, etas=True, Mmax=Mmax):
    (delta_time, magnitude, index, parent_index) = event
    print('dt=%.2f days, M=%.2f, i=%d, parent=%d' % event)

In [ ]:
gr_omlaw.Mc = 0.7
syncat = gr_omlaw.gen_aftershock_catalog(duration, mainshock=mainshock, etas=False, Mmax=Mmax)
syncat.print_list()